In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 https://develo

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-02 20:17:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-10-02 20:17:37 (11.7 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Review-Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://amazon-officereviews.s3.amazonaws.com/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Office_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.|2015-08-31 00:00:00|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| 

In [36]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

vine1_df = df.withColumn("star_rating",col("star_rating").cast("Integer"))
vine1_df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'timestamp')]

In [37]:
# Create the vine_table DataFrame
vine_df = vine1_df.select(["review_id", "star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R18RVCKGH1SSI9|          5|            0|          0|   N|                Y|
|R3L4L6LW1PUOFY|          5|            0|          1|   N|                Y|
|R2J8AWXWTDX2TF|          5|            0|          0|   N|                Y|
|R1PR37BR7G3M6A|          1|            2|          3|   N|                Y|
|R3BDDDZMZBZDPU|          4|            0|          0|   N|                Y|
| R8T6MO75ND212|          5|            0|          0|   N|                Y|
|R2YWMQT2V11XYZ|          5|            0|          0|   N|                N|
|R1V2HYL6OI9V39|          5|            6|          6|   N|                Y|
|R3BLQBKUNXGFS4|          5|            0|          0|   N|                Y|
|R17MOWJCAR9Y8Q|          5|            0|          0|   N|     

In [52]:
# Create new table with total votes equal or greater than 20
total_votes20_df = vine_df.filter('total_votes>=20')
total_votes20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|
| RLV4XQ49EBYYU|          1|           56|         65|   N|                Y|
| RPBPAJKFD0T26|          5|           63|         63|   N|                Y|
|R1IHO8C56KBN6C|          1|           61|         64|   N|     

In [40]:
# Create DF of table where the # of helpful_votes/total_votes >= 50%
helpful_votes_df = total_votes20_df.filter("helpful_votes/total_votes>=.50")
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|
| RLV4XQ49EBYYU|          1|           56|         65|   N|                Y|
| RPBPAJKFD0T26|          5|           63|         63|   N|                Y|
|R1IHO8C56KBN6C|          1|           61|         64|   N|     

In [41]:
# Create table where the review is part of the Vine program (paid)
vine_reviews_df = helpful_votes_df.filter(helpful_votes_df["vine"] == 'Y')
vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3JIGR122X50ZV|          5|         2280|       2314|   Y|                N|
|R3KV8P8WLD2KKC|          5|          263|        286|   Y|                N|
|R2LFY2ZPJ3J7RH|          4|           36|         38|   Y|                N|
|R3P01OXMA0U0CA|          5|          114|        117|   Y|                N|
|R2SCMXBD9OQP8Z|          2|           22|         26|   Y|                N|
|R2FGS9ECOK5IQN|          5|           22|         23|   Y|                N|
| R1V9CO41XS6QY|          2|           56|         58|   Y|                N|
|R1LEP1JGXRY2YT|          2|           18|         20|   Y|                N|
|R1ZWQJ3ZBSIZ1K|          4|           19|         21|   Y|                Y|
| RX1G4FUE5HZGN|          3|           52|         58|   Y|     

In [42]:
# Create table where the review is NOT part of the Vine program (Nonpaid)
non_vine_reviews_df = helpful_votes_df.filter(helpful_votes_df["vine"] == 'N')
non_vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|
| RLV4XQ49EBYYU|          1|           56|         65|   N|                Y|
| RPBPAJKFD0T26|          5|           63|         63|   N|                Y|
|R1IHO8C56KBN6C|          1|           61|         64|   N|     

In [43]:
from pyspark.sql.functions import count

In [44]:
# Total # of paid reviews:
vine_reviews_count = vine_reviews_df.count()
vine_reviews_count

969

In [45]:
# Total # of NON paid reviews:
nonvine_reviews_count = non_vine_reviews_df.count()
nonvine_reviews_count

43745

In [46]:
# Total Number of 5-star reviews:
reviews_5stars = vine_df.filter("star_rating == 5").count()
reviews_5stars

1584177

In [47]:
# Count Helpful votes
helpful_review_count = helpful_votes_df.count()
helpful_review_count

44714

In [48]:
# Number of paid 5-star reviews:
paid_reviews_5stars = vine_reviews_df.filter("star_rating == 5").count()
paid_reviews_5stars

430

In [49]:
# Number of Nonpaid 5-star reviews:
nonpaid_reviews_5stars = non_vine_reviews_df.filter("star_rating == 5").count()
nonpaid_reviews_5stars

19233

In [50]:
# % of paid 5star reviews:
perc_paid_5stars = (paid_reviews_5stars/vine_reviews_count)*100
perc_paid_5stars

44.375644994840044

In [51]:
# % of nonpaid 5star reviews:
perc_nonpaid_5stars = (nonpaid_reviews_5stars/nonvine_reviews_count)*100
perc_nonpaid_5stars

43.96616756200709

In [53]:
# Total for overall reviews:
overall_reviews_count = vine_df.count()
overall_reviews_count

2642434

In [56]:
# Percentage of paid 5-stars reviews vs. Overall reviews
overall_perc_paid_reviews = round((paid_reviews_5stars/overall_reviews_count)*100,2)
overall_perc_paid_reviews

0.02

In [57]:
# Percentage of paid 5-stars reviews vs. Overall reviews
overall_perc_nonpaid_reviews = round((nonpaid_reviews_5stars/overall_reviews_count)*100,2)
overall_perc_nonpaid_reviews

0.73

In [61]:
# Create a new DataFrame to summarize all information:

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

finaldf = [
    ("Overall Reviews", overall_reviews_count),
    ("Helpful Reviews", helpful_review_count),
    ("Total Paid Reviews", vine_reviews_count),
    ("Total Unpaid Reviews", nonvine_reviews_count),
    ("Paid Five Star Reviews", paid_reviews_5stars),
    ("Unpaid Five Star Reviews", nonpaid_reviews_5stars),
    ("Percentage of Paid 5-star Reviews", round(perc_paid_5stars, 2)),
    ("Percentage of Unpaid 5-star Reviews", round(perc_nonpaid_5stars, 2)),
    ("Paid reviews Overall percentage", overall_perc_paid_reviews),
    ("NOT Paid reviews Overall percentage", overall_perc_nonpaid_reviews)
]

schema = StructType([
    StructField("Description", StringType(), True),\
    StructField("Count/Percent", StringType(), True),\
])

Summary_df = spark.createDataFrame(data=finaldf, schema=schema) 
Summary_df.printSchema()
Summary_df.show(truncate=False)

root
 |-- Description: string (nullable = true)
 |-- Count/Percent: string (nullable = true)

+-----------------------------------+-------------+
|Description                        |Count/Percent|
+-----------------------------------+-------------+
|Overall Reviews                    |2642434      |
|Helpful Reviews                    |44714        |
|Total Paid Reviews                 |969          |
|Total Unpaid Reviews               |43745        |
|Paid Five Star Reviews             |430          |
|Unpaid Five Star Reviews           |19233        |
|Percentage of Paid 5-star Reviews  |44.38        |
|Percentage of Unpaid 5-star Reviews|43.97        |
|Paid reviews Overall percentage    |0.02         |
|NOT Paid reviews Overall percentage|0.73         |
+-----------------------------------+-------------+

